<a href="https://colab.research.google.com/github/alessandromazza93/SDM_2024/blob/main/ReinforcementLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Initialize Python
#------------------------------------------------
#           Play around with these
#------------------------------------------------
alpha = 0.1  # Learning rate [0 = no influence of PEs, 1 = total influence]
beta = 0.05  # Inverse temperature [0 = deterministic, Inf = random behavior]
Qzero_A = 0.5  # Initial values for option A
Qzero_B = 0.5  # Initial values for options B
#------------------------------------------------








# Set pre-defined outcomes
nTrials = 18  # Number of trials
reward_goes_to = np.array(["A", "A", "B", "A", "A", "B", "B", "A", "A", "A",
                           "A", "A", "A", "B", "A", "A", "B", "A"])
# pRewA = 0.75
# reward_goes_to = np.array(["A" if np.random.rand() <= pRewA else "B" for _ in range(nTrials)])

# Simulate data
Q = np.array([[Qzero_A, Qzero_B]])
P = []
choices = []
rewards = []

for t in range(nTrials):
    # Compute choice probabilities [Softmax]
    #--------------------------------------------
    prob_A = np.exp(Q[t, 0] / beta) / (np.exp(Q[t, 0] / beta) + np.exp(Q[t, 1] / beta))  # P(A)
    prob_B = np.exp(Q[t, 1] / beta) / (np.exp(Q[t, 0] / beta) + np.exp(Q[t, 1] / beta))  # P(B)
    P.append([prob_A, prob_B])

    # Simulate choice based on probabilities
    #---------------------------------------
    if np.random.rand() < prob_A:
        choices.append("A")  # Choose option A
    else:
        choices.append("B")  # Choose option B

    # Get outcome for the chosen option
    #------------------------------------
    if choices[-1] == reward_goes_to[t]:
        rewards.append(1)
    else:
        rewards.append(0)

    # Update values [Rescorla-Wagner rule]
    #------------------------------------------
    if choices[-1] == "A":
        Q_t1_A = Q[t, 0] + alpha * (rewards[-1] - Q[t, 0])  # Update value for option A
        Q_t1_B = Q[t, 1]  # Keep same value for option B
    elif choices[-1] == "B":
        Q_t1_B = Q[t, 1] + alpha * (rewards[-1] - Q[t, 1])  # Update value for option B
        Q_t1_A = Q[t, 0]  # Keep same value for option A

    Q = np.vstack([Q, [Q_t1_A, Q_t1_B]])  # Append updated values

# Cut n+1th trial
Q = Q[:-1, :]
P = np.array(P)

# Plot values over time
plt.figure(figsize=(10, 8))
plt.subplot(211)
plt.plot(range(1, nTrials + 1), Q[:, 0], '-o', linewidth=2, label='Q(A)')
plt.plot(range(1, nTrials + 1), Q[:, 1], '-o', linewidth=2, label='Q(B)')
plt.xlabel('Trial')
plt.ylabel('Subjective value')
plt.legend(loc='upper left')
plt.title('Choice Values')
plt.ylim([0, 1])

# Plot probabilities over time
plt.subplot(212)
plt.plot(range(1, nTrials + 1), P[:, 0], '-o', linewidth=2, label='P(A)')
plt.plot(range(1, nTrials + 1), P[:, 1], '-o', linewidth=2, label='P(B)')
plt.xlabel('Trial')
plt.ylabel('Probability')
plt.legend(loc='upper left')
plt.axhline(1.0, color='gray', linestyle='--', alpha=0.5)
for i, choice in enumerate(choices):
    plt.axvline(x=i + 1, color='blue' if choice == "A" else 'red', linestyle='--', alpha=0.5)

reward_indices = np.array(rewards) == 1
non_reward_indices = np.array(rewards) == 0
plt.scatter(np.where(reward_indices)[0] + 1, 1.1 * np.ones(np.sum(reward_indices)), color='green', label='Reward', zorder=5)
plt.scatter(np.where(non_reward_indices)[0] + 1, 1.1 * np.ones(np.sum(non_reward_indices)), color='red', label='No Reward', zorder=5)
plt.title('Choice Probabilities')
plt.ylim([-0.1, 1.1])
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()
